In [2]:
from bs4 import BeautifulSoup
import pandas as pd
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

opts = Options()
opts.add_experimental_option('prefs', {'intl.accept_languages': 'en,en_US'})
driver = webdriver.Chrome(options=opts)
driver.minimize_window()
print(f'Chrome Web Driver loaded. Version: {driver.capabilities["browserVersion"]}\n')  # use "version" on Linux


def request_page(url):
    """ Returns BeautifulSoup4 Objects (soup)"""
    driver.get(url)
    return BeautifulSoup(driver.page_source, "html.parser")


URLS = {
    # 'P1-MA': 'https://www.singaporemathguru.com/module2?cr=07OeW7mqKNGdUhVv6Tth',
    #     'P2-MA': 'https://www.singaporemathguru.com/module2?cr=1BPikqvu1N9jXPXyQEyf',
        'P3-MA': 'https://www.singaporemathguru.com/module2?cr=0LX7CC6GLcQzQXo2XXJ2',
        'P4-MA': 'https://www.singaporemathguru.com/module2?cr=JHiNPh7RNFy3gQEgLZo0',
        'P5-MA': 'https://www.singaporemathguru.com/module2?cr=WlbWcVqYOrG64YHqSNDv',}
        #'P6-MA': 'https://www.singaporemathguru.com/module2?cr=9jN6iUGLBbnpvqPLYOiB'}




def scrap(lvl, URL):
    soup = request_page(URL)
    exercises = soup.find_all("ul", class_="exercise")

    chapter_names = []
    qns_with_label = []
    domain = 'https://www.singaporemathguru.com/'
    for exercise in exercises:
        chapter_name = ' '.join(exercise.find('li').find('span').get_text().strip().split()[:-2])
        chapter_names.append(chapter_name)
        qns_lists = exercise.find_all("li", class_="")
        for qns_list in qns_lists:
            for i in qns_list.find_all('a', class_=""):
                qns_with_label.append({'chapter_name': chapter_name, 'link': domain+i['href']})

    print(f'Total {len(qns_with_label)} questions. scraping now...')
    for i in range(len(qns_with_label)):
        question_soup = request_page(qns_with_label[i]['link'])
        question = question_soup.find('div', class_="span10")
        question_str = ' '.join([p.get_text(strip=True) for p in question.find_all('p', class_='')])
        ans = ' '.join([p.get_text(strip=True) for p in question.find_all('div', id='div_view_answer')]).split(':')[-1]
        qns_with_label[i]['qns'] = question_str
        qns_with_label[i]['ans'] = ans

    df = pd.DataFrame.from_dict(qns_with_label)
    df.to_csv(f'{lvl}.csv', index=False)

for i, v in URLS.items():
    print(f'Scraping {i}...')
    scrap(i, v)
    print(f'{i} scraped.')

Chrome Web Driver loaded. Version: 96.0.4664.110

Scraping P3-MA...
Total 1364 questions. scraping now...
P3-MA scraped.
Scraping P4-MA...
Total 334 questions. scraping now...
P4-MA scraped.
Scraping P5-MA...
Total 698 questions. scraping now...
P5-MA scraped.
